<a href="https://colab.research.google.com/github/Proxxied/MovieRec/blob/main/Using_100K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl http://files.grouplens.org/datasets/movielens/ml-100k.zip -o ml-100k.zip
!pip install scikit-surprise
!pip install recmetrics

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4808k  100 4808k    0     0  10.4M      0 --:--:-- --:--:-- --:--:-- 10.3M


In [2]:
import zipfile
with zipfile.ZipFile('ml-100k.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [3]:
import pandas as pd
import numpy as np

r1cols = ['user_id', 'movie_id', 'rating', 'timestamp']
r2cols = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama',  'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

moviesdf = pd.read_csv('data/ml-100k/u.item', sep='|' , names=r2cols, encoding='latin-1')
ratingsdf = pd.read_csv('data/ml-100k/u.data', sep='\t', names=r1cols, encoding='latin-1')
ratingsbase = pd.read_csv('data/ml-100k/u1.base', sep='\t', names=r1cols, encoding='latin-1')
ratingstest = pd.read_csv('data/ml-100k/u1.test', sep='\t', names=r1cols, encoding='latin-1')

In [4]:
#TRAIN
totuserbase = ratingsbase['user_id'].unique().max()
totmoviebase = ratingsbase['movie_id'].unique().max()
totuserbase, totmoviebase
totusertest = ratingstest['user_id'].unique().max()
totmovietest = ratingstest['movie_id'].unique().max()
totusertest, totmovietest

trainmat = np.zeros((totuserbase, totmoviebase))
for line in ratingsbase.itertuples():
  trainmat [line[1]-1,line[2]-1] = line[3]
  testmat = np.zeros((totusertest, totmovietest))
for line in ratingstest.itertuples():
   testmat[line[1]-1,line[2]-1] = line[3]

In [5]:
#TRAIN
import matplotlib.pyplot as plt
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratingsdf[['user_id', 'movie_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.20)
algo = SVD()
algo.fit(trainset)

test = algo.test(testset)
test = pd.DataFrame(test)
test.drop("details", inplace=True, axis=1)
test.columns = ['user_id', 'movie_id', 'true', 'preds']
test.head(10) 

,user_id,movie_id,true,preds
0,280,586,4.0,2.542399
1,619,50,4.0,4.399237
2,605,1040,2.0,2.516849
3,877,463,4.0,3.897850
4,899,655,4.0,3.531560
5,761,7,4.0,3.020400
6,716,655,4.0,3.937695
7,542,265,4.0,3.583466
8,391,491,3.0,3.612618
9,604,185,2.0,3.670699


USER-BASED COLLABORATIVE FILTERING

In [6]:
from sklearn.metrics.pairwise import pairwise_distances
import math

#USER-BASED COLLABORATIVE FILTERING

usersim = pairwise_distances(trainmat, metric='cosine')
print('shape: ', usersim.shape)

shape:  (943, 943)


In [7]:
def predictuser(trainmat, usersim, n_sim = 50):
    
    sim_n = usersim.argsort()[:,-n_sim:][:,::-1]
    preds = np.zeros((totuserbase,totmoviebase))
    
    for i,users in enumerate(sim_n):
        simuser = users
        sim_n = usersim[i,simuser]
        matrixn = trainmat[simuser,:]
        ratedmovies = sim_n[:,np.newaxis].T.dot(matrixn - matrixn.mean(axis=1)[:,np.newaxis])/ sim_n.sum()
        preds[i,:]  = ratedmovies
        
    return preds

In [8]:
predictions = predictuser(trainmat,usersim, 50) + trainmat.mean(axis=1)[:, np.newaxis]
print('predictions shape ',predictions.shape)

predictions

predictions shape  (943, 1682)


array([[ 0.25719463,  0.25719463,  0.25719463, ...,  0.25719463,
         0.25719463,  0.25719463],
       [ 0.37279746,  0.05657032, -0.02277058, ..., -0.02277058,
        -0.02277058, -0.02277058],
       [ 0.8756956 ,  0.0156956 ,  0.0156956 , ..., -0.0443044 ,
        -0.0443044 , -0.0443044 ],
       ...,
       [-0.00687779,  0.0531826 , -0.00687779, ..., -0.00687779,
        -0.00687779, -0.00687779],
       [ 1.10227848,  0.14145999,  0.25983283, ...,  0.14145999,
         0.14145999,  0.14145999],
       [ 0.28416171,  0.28416171,  0.28416171, ...,  0.28416171,
         0.28416171,  0.28416171]])

In [9]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import recall_score

predictedratings = predictions[testmat.nonzero()]
testtrue = testmat[testmat.nonzero()]

#RMSE TEST

print('\nRMSE: {} \n'.format(math.sqrt(mean_squared_error(predictedratings,testtrue))))

#RECALL TEST
y_true = np.int64(testtrue)
y_pred = np.int64(test.preds)
print('\nRECALL: {} \n'.format(recall_score(y_true, y_pred, average=None)))



RMSE: 3.445305491245221 


RECALL: [0.         0.01869159 0.16149635 0.55113856 0.24431986 0.00358986] 



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
movie_id = 1
userratings = predictions[movie_id-1,:]
trainunknown = np.where(trainmat[movie_id-1,:] == 0)[0]
trainunknown
rec = userratings[trainunknown]
rec.shape

print('\nMovie {} other similar movies are: \n'.format(movie_id))

for movie_id in rec.argsort()[-5:][: : -1]:
    print(movie_id +1)


Movie 1 other similar movies are: 

165
153
178
151
129


ITEM-BASED COLLABORATIVE FILTERING

In [11]:
moviesim = pairwise_distances(trainmat.T, metric = 'cosine')
moviesim

array([[0.        , 0.64235362, 0.69061028, ..., 1.        , 0.94896481,
        0.94896481],
       [0.64235362, 0.        , 0.77839991, ..., 1.        , 0.91221914,
        0.91221914],
       [0.69061028, 0.77839991, 0.        , ..., 1.        , 1.        ,
        0.89333228],
       ...,
       [1.        , 1.        , 1.        , ..., 0.        , 1.        ,
        1.        ],
       [0.94896481, 0.91221914, 1.        , ..., 1.        , 0.        ,
        1.        ],
       [0.94896481, 0.91221914, 0.89333228, ..., 1.        , 1.        ,
        0.        ]])

In [12]:
def predictmovies(trainmat, moviesim, n_sim=30):
    
    sim_n = moviesim.argsort()[:,-n_sim:][:,::-1]
    print('sim_n shape: ', sim_n.shape)
    preds = np.zeros((totuserbase,totmoviebase))
    
    
    for i,movie in enumerate(sim_n):
        simmovies = movie
        sim_n = moviesim[i,simmovies]
        matrix_n = trainmat[:,simmovies]
        ratedmovies = matrix_n.dot(sim_n)/sim_n.sum()
        preds[:,i]  = ratedmovies
        
    return preds

In [13]:
predictions = predictmovies(trainmat,moviesim,50)
print('predictions shape ', predictions.shape)
predictions

sim_n shape:  (1682, 50)
predictions shape  (943, 1682)


array([[0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.24],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.26, 0.08, 0.18],
       [0.  , 0.  , 0.06, ..., 0.46, 0.32, 0.14]])

In [14]:
predictedratings = predictions[testmat.nonzero()]
testtrue = testmat[testmat.nonzero()]

#RMSE TEST

print('\nRMSE: {} \n'.format(math.sqrt(mean_squared_error(predictedratings,testtrue))))

#RECALL TEST
y_true = np.int64(testtrue)
y_pred = np.int64(test.preds)
print('\nRECALL: {} \n'.format(recall_score(y_true, y_pred, average=None)))


RMSE: 3.6988157672422672 


RECALL: [0.         0.01869159 0.16149635 0.55113856 0.24431986 0.00358986] 



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
user_id = 69
userratings = predictions[user_id-1,:]
trainunknown = np.where(trainmat[user_id-1,:] == 0)[0]
trainunknown
rec = userratings[trainunknown]
rec.shape

print('\nUser {} recommendations are: \n'.format(user_id))

for movie_id in rec.argsort()[-5:][: : -1]:
    print(movie_id +1)


User 69 recommendations are: 

1411
1019
1211
758
961


CONTENT-BASED FILTERING

In [16]:
contentdf=moviesdf[['movie_title','release_date']]
contentdf.head()

,movie_title,release_date
0,Toy Story (1995),01-Jan-1995
1,GoldenEye (1995),01-Jan-1995
2,Four Rooms (1995),01-Jan-1995
3,Get Shorty (1995),01-Jan-1995
4,Copycat (1995),01-Jan-1995


In [17]:
contentdf.reset_index(inplace=True)
contentdf.head()

,index,movie_title,release_date
0,0,Toy Story (1995),01-Jan-1995
1,1,GoldenEye (1995),01-Jan-1995
2,2,Four Rooms (1995),01-Jan-1995
3,3,Get Shorty (1995),01-Jan-1995
4,4,Copycat (1995),01-Jan-1995


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tf=TfidfVectorizer(stop_words='english')
tf_mat=tf.fit_transform(contentdf['movie_title'].fillna(''))
cosinesim = cosine_similarity(tf_mat)

#IDENTIFY MOVIE ID HERE:
contentdf[contentdf['movie_title']=='Toy Story (1995)']

,index,movie_title,release_date
0,0,Toy Story (1995),01-Jan-1995


In [19]:
#USE MOVIE ID TO CHANGE COSINE SIM NUM:
sim=cosinesim[0].tolist()
ind=[]
val=[]
for i in range(len(sim)):
    ind.append(i)
    val.append(sim[i])
    dic={'index':ind,'value':val}
rec_df=pd.DataFrame(dic) 
rec=rec_df.merge(contentdf,on='index')
rec.head()

,index,value,movie_title,release_date
0,0,1.000000,Toy Story (1995),01-Jan-1995
1,1,0.088473,GoldenEye (1995),01-Jan-1995
2,2,0.092783,Four Rooms (1995),01-Jan-1995
3,3,0.088473,Get Shorty (1995),01-Jan-1995
4,4,0.088473,Copycat (1995),01-Jan-1995


In [20]:
print(rec.sort_values(by='value',ascending=False).head(50)['movie_title'])

0                                        Toy Story (1995)
1071                    Pyromaniac's Love Story, A (1995)
1343                         Story of Xinghua, The (1993)
477                        Philadelphia Story, The (1940)
547                     NeverEnding Story III, The (1994)
307                        FairyTale: A True Story (1997)
1546                                     Show, The (1995)
1589                               To Have, or Not (1995)
1052                                  Now and Then (1995)
1652    Entertaining Angels: The Dorothy Day Story (1996)
598     Police Story 4: Project S (Chao ji ji hua) (1993)
722                               Boys on the Side (1995)
938                            Murder in the First (1995)
1519                                     Fear, The (1995)
555                                      Wild Bill (1995)
1146                                     My Family (1995)
714                                     To Die For (1995)
1598          

In [21]:
#IMPORT FOR VISUALISATIONS - IGNORE
import pandas as pd
df = pd.read_csv('data/ml-100k/u.data', sep='\t', names=r1cols, encoding='latin-1')
df.to_excel("ratings.xlsx")  

df1 = pd.read_csv('data/ml-100k/u.item', sep='|' , names=r2cols, encoding='latin-1')
df1.to_excel("movies.xlsx")  

r3cols = ['genre', 'quantity']

df2 = pd.read_csv('data/ml-100k/u.genre', sep='|', names=r3cols, encoding='latin-1')
df2.to_excel("genres.xlsx")  
#df2.head()